In [1]:
from pymatgen.io.vasp import Poscar, Xdatcar
from pymatgen.symmetry.groups import SpaceGroup
import numpy as np
from site_analysis import Atom, Analysis, PolyhedralSite
from site_analysis.tools import get_nearest_neighbour_indices
from collections import Counter
import tqdm
import matplotlib.pyplot as plt

In [2]:
directory = '../../data'
structure = Poscar.from_file(f'{directory}/reference_structures/Li6PS5I_alltet_sites.POSCAR.vasp').structure

In [3]:
from pymatgen import Structure
lattice = structure.lattice
t0 = Structure.from_spacegroup(sg='F-43m', lattice=lattice, species=['P'], coords=[[0.5, 0.0, 0.0]])
t1 = Structure.from_spacegroup(sg='F-43m', lattice=lattice, species=['Li'], coords=[[0.9, 0.9, 0.6]])
t2 = Structure.from_spacegroup(sg='F-43m', lattice=lattice, species=['Li'], coords=[[0.23, 0.92, 0.08]])
t3 = Structure.from_spacegroup(sg='F-43m', lattice=lattice, species=['Li'], coords=[[0.25, 0.25, 0.25]])
t4 = Structure.from_spacegroup(sg='F-43m', lattice=lattice, species=['Li'], coords=[[0.15, 0.15, 0.15]])
t5 = Structure.from_spacegroup(sg='F-43m', lattice=lattice, species=['Li'], coords=[[0.0, 0.183, 0.183]])
tet_sites = [t0, t1, t2, t3, t4, t5]

In [62]:
def tetrahedral_analysis( xdatcar, x_spec ):
    md_structure = xdatcar.structures[0]
    
    Atom.reset_index()
    atoms = [Atom(species_string='Li') for site in md_structure if site.species_string is 'Li']

    t0_indices = get_nearest_neighbour_indices( md_structure, t0*[2,2,2], vertex_species=['S', x_spec], n_coord=4)
    t1_indices = get_nearest_neighbour_indices( md_structure, t1*[2,2,2], vertex_species=['S', x_spec], n_coord=4)
    t2_indices = get_nearest_neighbour_indices( md_structure, t2*[2,2,2], vertex_species=['S', x_spec], n_coord=4)
    t3_indices = get_nearest_neighbour_indices( md_structure, t3*[2,2,2], vertex_species=['S', x_spec], n_coord=4)
    t4_indices = get_nearest_neighbour_indices( md_structure, t4*[2,2,2], vertex_species=['S', x_spec], n_coord=4)
    t5_indices = get_nearest_neighbour_indices( md_structure, t5*[2,2,2], vertex_species=['S', x_spec], n_coord=4)
    
    PolyhedralSite.reset_index()

    t0_sites = [ PolyhedralSite(vertex_species=['S', x_spec], vertex_indices=vi, 
                 label='0') for vi in t0_indices ]
    t1_sites = [ PolyhedralSite(vertex_species=['S', x_spec], vertex_indices=vi, 
                 label='1') for vi in t1_indices ]
    t2_sites = [ PolyhedralSite(vertex_species=['S', x_spec], vertex_indices=vi, 
                 label='2') for vi in t2_indices ]
    t3_sites = [ PolyhedralSite(vertex_species=['S', x_spec], vertex_indices=vi, 
                 label='3') for vi in t3_indices ]
    t4_sites = [ PolyhedralSite(vertex_species=['S', x_spec], vertex_indices=vi, 
                 label='4') for vi in t4_indices ]
    t5_sites = [ PolyhedralSite(vertex_species=['S', x_spec], vertex_indices=vi, 
                 label='5') for vi in t5_indices ]
    sites = t0_sites + t1_sites + t2_sites + t3_sites + t4_sites + t5_sites

    analysis = Analysis(atoms=atoms, sites=sites)
    return analysis

def site_populations(analysis):
    c = Counter()
    for site in analysis.sites:
        c[site.label] += len([ 1 for ts in site.trajectory if len(ts)>0 ])
    total = sum(c.values())
    return {label: n/total for label, n in c.items()}

def coordination_dependent_site_populations(analysis, ref_structure):
    c = { str(i): Counter() for i in range(6) }
    s = { str(i): Counter() for i in range(6) }
    for site in analysis.sites:
        n_halogen = len([ 1 for spec in site.get_vertex_species(ref_structure) if spec in ['Cl', 'I']])
        c[site.label][n_halogen] += len([ 1 for ts in site.trajectory if len(ts)>0 ])
        s[site.label][n_halogen] += 1
    return c, s

In [54]:
populations = {'Li6PS5I': {}, 'Li6PS5Cl': {}}

In [57]:
system = 'Li6PS5I'
disorder = '0p'
x_spec = 'I'
xdatcar = Xdatcar(f'{directory}/{system}/{disorder}/run1/inherent_XDATCAR.gz')
analysis = tetrahedral_analysis( xdatcar, x_spec )
analysis.trajectory_from_structures( xdatcar.structures, progress='notebook')
coordination_dependent_site_populations( analysis, xdatcar.structures[0])
# populations[system][disorder] = site_populations( analysis )

{'0': Counter(), '1': Counter(), '2': Counter(), '3': Counter(), '4': Counter(), '5': Counter()}
[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
{'0': Counter(), '1': Counter(), '2': Counter(), '3': Counter(), '4': Counter(), '5': Counter()}
0
len 0
before 0 Counter()
after 0 Counter({0: 0})
[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [

len 0
before 2 Counter({1: 205})
after 2 Counter({1: 205})
[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
{'0': Counter({0: 0}), '1': Counter({0: 0}), '2': Counter({1: 205}), '3': Counter(), '4': Counter(), '5': Counter()}
1
len 0
before 2 Counter({1: 205})
after 2 Counter({1: 205})
[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [

{'0': Counter({0: 0}), '1': Counter({0: 0}), '2': Counter({1: 659}), '3': Counter({0: 0}), '4': Counter(), '5': Counter()}
0
len 0
before 3 Counter({0: 0})
after 3 Counter({0: 0})
[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
{'0': Counter({0: 0}), '1': Counter({0: 0}), '2': Counter({1: 659}), '3': Counter({0: 0}), '4': Counter(), '5': Counter()}
0
len 0
before 3 Counter({0: 0})
after 3 Counter({0: 0})
[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []

after 5 Counter({1: 8007})
[[], [], [], [88], [88], [88], [88], [88], [88], [], [], [], [88], [88], [88], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [88], [88], [88], [88], [88], [88], [88], [], [88], [88], [88], [88], [88], [88], [88], [88], [88], [], [], [], [88], [88], [88], [88], [88], [88], [88], [88], [88], [], [], [88], [88], [88], [88], [88], [88], [88], [88], [88], [88], [88], [88], [], [], [], [], [], [88], [88], [88], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [88], [88], [88], [88], [88], [88], [], [], [], [], [], [], [], [], [88], [88], [], [], [], [], [], [], [], [88], [88], [88], [88]]
{'0': Counter({0: 0}), '1': Counter({0: 0}), '2': Counter({1: 659}), '3': Counter({0: 0}), '4': Counter({1: 0}), '5': Counter({1: 8007})}
1
len 61
before 5 Counter({1: 8007})
after 5 Counter({1: 8068})
[[105], [105], [105], [105], [105], [105], [], [], [], [], [], [], [105], [105], [105], 

In [63]:
system = 'Li6PS5I'
disorder = '50p'
x_spec = 'I'
xdatcar = Xdatcar(f'{directory}/{system}/{disorder}/run1/inherent_XDATCAR.gz')
analysis = tetrahedral_analysis( xdatcar, x_spec )
analysis.trajectory_from_structures( xdatcar.structures, progress='notebook')
coordination_dependent_site_populations( analysis, xdatcar.structures[0])
# populations[system][disorder] = site_populations( analysis )


({'0': Counter({0: 0}),
  '1': Counter({1: 0, 0: 0}),
  '2': Counter({2: 957, 1: 3245, 0: 1320}),
  '3': Counter({0: 0}),
  '4': Counter({1: 11, 0: 103}),
  '5': Counter({1: 7410, 2: 2511, 0: 3643})},
 {'0': Counter({0: 32}),
  '1': Counter({1: 64, 0: 64}),
  '2': Counter({2: 96, 1: 192, 0: 96}),
  '3': Counter({0: 32}),
  '4': Counter({1: 64, 0: 64}),
  '5': Counter({1: 192, 2: 96, 0: 96})})

In [ ]:
for s in analysis.sites:
    print( [ site.species_string for i, site in enumerate(xdatcar.structures[0]) if i in s.vertex_indices ] )
    print( s.label)

In [ ]:
system = 'Li6PS5I'
disorder = '100p'
x_spec = 'I'
xdatcar = Xdatcar(f'{directory}/{system}/{disorder}/run1/inherent_XDATCAR.gz')
analysis = tetrahedral_site_analysis( xdatcar, x_spec )
populations[system][disorder] = site_populations( analysis )

In [ ]:
system = 'Li6PS5Cl'
disorder = '0p'
x_spec = 'Cl'
xdatcar = Xdatcar(f'{directory}/{system}/{disorder}/run1/inherent_XDATCAR.gz')
analysis = tetrahedral_site_analysis( xdatcar, x_spec )
populations[system][disorder] = site_populations( analysis )

In [ ]:
system = 'Li6PS5Cl'
disorder = '50p'
x_spec = 'Cl'
xdatcar = Xdatcar(f'{directory}/{system}/{disorder}/run1/inherent_XDATCAR.gz')
analysis = tetrahedral_site_analysis( xdatcar, x_spec )
populations[system][disorder] = site_populations( analysis )

In [ ]:
system = 'Li6PS5Cl'
disorder = '100p'
x_spec = 'Cl'
xdatcar = Xdatcar(f'{directory}/{system}/{disorder}/run1/inherent_XDATCAR.gz')
analysis = tetrahedral_site_analysis( xdatcar, x_spec )
populations[system][disorder] = site_populations( analysis )

In [ ]:
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.sans-serif'] = ['Minion Pro']
rcParams['pdf.fonttype'] = 42
rcParams['ps.fonttype'] = 42
rcParams.update({'font.size': 18})
rcParams['mathtext.fontset'] = 'dejavuserif'

In [ ]:
ind = np.arange(3)
fig, ax = plt.subplots(2,1, figsize=(7,7))
title = { 'Li6PS5I': r'Li$_6$PS$_5$I',
          'Li6PS5Cl': r'Li$_6$PS$_5$Cl'}

color = {'5': '#4179a7',
         '2': '#f28e2b',
         '4': '#59a14f',
         '3': '#ef9da7', 
         '1': '#bab0ac',
         '0': '#9f9f9f'}

for j, system in enumerate(['Li6PS5I', 'Li6PS5Cl']):
    bottom = np.zeros(3)
    for i in range(5,-1,-1):
        heights = [ populations[system]['0p'][str(i)],
                    populations[system]['50p'][str(i)],
                    populations[system]['100p'][str(i)] ]

        heights.reverse()

        width=0.5
        ax[j].barh(ind, heights, width, left=bottom, label=f'{i}', color=color[str(i)])
        bottom += heights
    ax[j].set_xlabel(r'$P\,(\mathrm{site})$')
    ax[j].set_yticklabels(['100%', '50%', '0%'])
    ax[j].set_yticks([0,1,2])
    ax[j].yaxis.tick_right()
    ax[j].set_title(title[system], pad=15, loc='left', fontsize=18)
ax[1].legend(bbox_to_anchor=(1.1,-0.6), ncol=6, handlelength=1, fancybox=False)
plt.tight_layout()
plt.savefig('../../figures/tetrahedral_site_populations.pdf', bbox_inches='tight')
plt.show()